In [1]:
from catinous.dataset import BatchDataset, ContinuousDataset
from catinous.dynamicmemory.DynamicMemoryModel import DynamicMemoryModel 
import catinous.dynamicmemory.DynamicMemoryModel as dmodel
import catinous.utils as cutils
import os
import pandas as pd

In [23]:
hparams = {
    'continuous': False,
    'datasetfile': '/project/catinous/cardiacslicessplit.csv',
    'batch_size': 16,
    'training_batch_size': 16,
    'task': 'cardiac',
    'model': 'fcn',
    'scanner': 'Siemens',
     'noncontinuous_steps': 500,
    'noncontinuous_train_splits': ['base', 'train'],
    'use_memory': False
}

val_dice1_Siemens = []
val_dice2_Siemens = []
val_dice3_Siemens = []
val_dice1_GE = []
val_dice1_Philips = []
val_dice1_Canon = []

for scanner in ['Siemens', 'GE', 'Philips', 'Canon', None]:
    hparams['scanner'] = scanner
    if scanner is None:
        hparams['order'] = ['Siemens', 'GE', 'Philips', 'Canon']
    #model, logs, df_cache = catsmodel.trained_model(hparams)
    expparams = cutils.default_params(DynamicMemoryModel.get_default_hparams(), hparams)
    exp_name = cutils.get_expname(expparams)

    max_version = max([int(x.split('_')[1]) for x in os.listdir(cutils.LOGGING_FOLDER + exp_name)])
    logs = pd.read_csv(cutils.LOGGING_FOLDER + exp_name + '/version_{}/metrics.csv'.format(max_version))
    
    val_dice1_Siemens.append(logs.iloc[-1].val_dice1_Siemens)
    val_dice1_GE.append(logs.iloc[-1].val_dice1_GE)
    val_dice1_Philips.append(logs.iloc[-1].val_dice1_Philips)
    val_dice1_Canon.append(logs.iloc[-1].val_dice1_Canon)

pd.DataFrame({'scanner': ['Siemens', 'GE', 'Philips', 'Canon', None], 'val_Siemens': val_dice1_Siemens, 
              'val_GE': val_dice1_GE, 'val_Philips': val_dice1_Philips, 'val_Canon': val_dice1_Canon})

,scanner,val_Siemens,val_GE,val_Philips,val_Canon
0,Siemens,0.807649,0.827656,0.843444,0.801174
1,GE,0.712092,0.826061,0.786773,0.774519
2,Philips,0.745954,0.782469,0.860170,0.828817
3,Canon,0.714380,0.752285,0.746388,0.878775
4,None,0.836838,0.848288,0.853259,0.875537


In [ ]:
hparams = {
    'continuous': False,
    'datasetfile': '/project/catinous/cardiacslicessplit.csv',
    'batch_size': 16,
    'training_batch_size': 16,
    'task': 'cardiac',
    'model': 'fcn',
    'scanner': 'Siemens',
     'noncontinuous_steps': 500,
    'noncontinuous_train_splits': ['base', 'train'],
    'use_memory': False
}

series_list = []

for scanner in ['Siemens', 'GE', 'Philips', 'Canon', None]:
    hparams['scanner'] = scanner
    if scanner is None:
        hparams['order'] = ['Siemens', 'GE', 'Philips', 'Canon']
    #model, logs, df_cache = catsmodel.trained_model(hparams)
    expparams = cutils.default_params(DynamicMemoryModel.get_default_hparams(), hparams)
    exp_name = cutils.get_expname(expparams)

    max_version = max([int(x.split('_')[1]) for x in os.listdir(cutils.LOGGING_FOLDER + exp_name)])
    logs = pd.read_csv(cutils.LOGGING_FOLDER + exp_name + '/version_{}/metrics.csv'.format(max_version))
    
    series = logs.iloc[-1]
    series['scanner'] = scanner
    
    series_list.append(series)

    
df_res = pd.DataFrame(series_list)
df_res = df_res.drop(columns=['created_at', 'epoch', 'train_loss'])
df_res

In [24]:
hparams = {
    'continuous': False,
    'datasetfile': '/project/catinous/cardiacslicessplit.csv',
    'batch_size': 16,
    'training_batch_size': 16,
    'task': 'cardiac',
    'model': 'fcn',
    'scanner': 'Siemens',
     'noncontinuous_steps': 500,
    'noncontinuous_train_splits': ['base'],
    'use_memory': False
}

expparams = cutils.default_params(DynamicMemoryModel.get_default_hparams(), hparams)
exp_name = cutils.get_expname(expparams)
max_version = max([int(x.split('_')[1]) for x in os.listdir(cutils.LOGGING_FOLDER + exp_name)])
logs = pd.read_csv(cutils.LOGGING_FOLDER + exp_name + '/version_{}/metrics.csv'.format(max_version))
logs.iloc[-1]  

created_at           2021-01-04 11:11:35.809529
epoch                                       NaN
train_loss                                  NaN
val_dice1_Canon                        0.795759
val_dice1_GE                           0.821302
val_dice1_Philips                       0.83066
val_dice1_Siemens                      0.820307
val_dice2_Canon                        0.622149
val_dice2_GE                           0.727665
val_dice2_Philips                      0.751006
val_dice2_Siemens                      0.710215
val_dice3_Canon                        0.580211
val_dice3_GE                           0.763741
val_dice3_Philips                      0.721128
val_dice3_Siemens                      0.767016
Name: 14, dtype: object

In [25]:
exp_name

'cardiac_batch_cardiacslicessplit_base_1_7826e2e7a0'